In [ ]:
from fastai.vision import *

In [ ]:
import cv2

In [ ]:
def readImage(path):
    img = cv2.imread(str(path))
    b, g, r = cv2.split(img)
    rgb_img = cv2.merge([r, g, b])
    return rgb_img/255.

In [ ]:
class MyImageItemList(ImageList):
    def open(self, fn:PathOrStr)->Image:
        img = readImage(fn)
        return vision.Image(px=pil2tensor(img, np.float32))

In [ ]:
df = pd.read_csv('../input/train_labels.csv')
df.head()

In [ ]:
bs = 128
sz = 90

In [ ]:
data = (MyImageItemList.from_csv(path='../input', csv_name='train_labels.csv', folder='train', suffix='.tif')
        .split_by_rand_pct(0.2)
        .label_from_df(cols='label')
        .add_test(MyImageItemList.from_folder('../input/test'))
        .transform(tfms=get_transforms(flip_vert=True, max_zoom=1.2, max_rotate=3), size=sz)
        .databunch(bs=bs)
        .normalize([tensor([0.702447, 0.546243, 0.696453]), 
                    tensor([0.238893, 0.282094, 0.216251])]))

In [ ]:
data.show_batch(rows=2, figsize=(4, 4))

In [ ]:
def getLearner():
    return cnn_learner(data, models.densenet169, pretrained=True, path='.', metrics=accuracy, 
                      ps=0.5, callback_fns=ShowGraph)
learn = getLearner()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(5, max_lr=1e-2, wd=1e-4)

In [ ]:
learn.recorder.plot_lr()

In [ ]:
learn.save('stage-1')

In [ ]:
learn.load('stage-1')

In [ ]:
learn.unfreeze()
learn.lr_find(wd=1e-4)
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(4, slice(2e-5, 3e-4), wd=1e-4)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(title='Confusion matrix')

In [ ]:
learn.save('stage-2')

In [ ]:
learn.load('stage-2')

In [ ]:
n_aug = 12
preds_n_avg = np.zeros((len(learn.data.test_ds.items), 2))
for n in range(n_aug):
    preds, y = learn.get_preds(ds_type=DatasetType.Test, with_loss=False)
    preds_n_avg = np.sum([preds_n_avg, preds.numpy()], axis=0)
preds_n_avg/=n_aug

In [ ]:
tumor_preds = preds_n_avg[:, 1]

In [ ]:
sub = pd.read_csv('../input/sample_submission.csv')

In [ ]:
sample_list = list(sub.id)
pred_list = [p for p in tumor_preds]
pred_dict = dict((key, value) for (key, value) in zip(learn.data.test_ds.items, pred_list))

In [ ]:
pred_dict

In [ ]:
pred_list_cor = [pred_dict[Path('../input/test/')/(id + '.tif')] for id in sample_list]
sub_df = pd.DataFrame({'id': sample_list, 'label': pred_list_cor})

In [ ]:
sub_df.head()

In [ ]:
sub_df.to_csv('prediction.csv', header=True, index=False)